## Loading data

In [1]:
# Loading the IPython extention autoreload.
%load_ext autoreload

In [2]:
# Reloading edited module without restarting the kernel.
# Referring to https://support.enthought.com/hc/en-us/articles/204469240-Jupyter-IPython-After-editing-a-module-changes-are-not-effective-without-kernel-restart
%autoreload

from utils import *

path_to_train_and_dev = '../../data/first-round/train_and_dev_set'
path_to_test = '../../data/first-round/test_set'
ratio_of_training_data = 0.8
train_set_x, train_set_y, dev_set_x, dev_set_y, test_set_x = load_data(path_to_train_and_dev, ratio_of_training_data, path_to_test)

In [3]:
print('train x:', len(train_set_x), 'train y', len(train_set_y), 'dev x:', len(
    dev_set_x), 'dev y:', len(dev_set_y), 'test x:', len(test_set_x))

train x: 32 train y 32 dev x: 8 dev y: 8 test x: 10


## Loading and compiling model

In [4]:
%autoreload

from model import *

# Creating model.
model = create_model(input_shape=(train_set_x.shape[1:]), summary=True)

# Instantiating the optimizer.
opt = Adam()

# Determining the loss and metric, and compiling the model.
model.compile(optimizer=opt, loss='mean_squared_error')

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 7500, 4)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 7500, 4)           16        
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               136192    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 137,489
Trainable params: 136,969
Non-trainable params: 520
_________________________________________________________________


In [5]:
#Loading weights, if needed.
#model.load_weights("./saved_weights/model-019epochs-0.028val_loss.h5")

## Fitting

In [6]:
# checkpoint
from keras.callbacks import ModelCheckpoint
filepath="./saved_weights/weights-{epoch:003d}epochs-{loss:.3f}loss-{val_loss:.3f}val_loss.h5"
checkpoint = ModelCheckpoint(filepath, save_weights_only=True)
callbacks_list = [checkpoint]

In [7]:
History_fit = model.fit(train_set_x, train_set_y, batch_size=train_set_x.shape[0], epochs=20, callbacks=callbacks_list, validation_data=(dev_set_x, dev_set_y))
history_fit = History_fit.history

Train on 32 samples, validate on 8 samples
Epoch 1/20
32/32 [==============================] - 27s 855ms/step - loss: 0.4003 - val_loss: 0.2769
Epoch 2/20
32/32 [==============================] - 24s 746ms/step - loss: 0.3822 - val_loss: 0.2600
Epoch 3/20
32/32 [==============================] - 24s 744ms/step - loss: 0.3625 - val_loss: 0.2586
Epoch 4/20
32/32 [==============================] - 26s 802ms/step - loss: 0.3440 - val_loss: 0.2544
Epoch 5/20
32/32 [==============================] - 24s 765ms/step - loss: 0.3203 - val_loss: 0.2556
Epoch 6/20
32/32 [==============================] - 24s 744ms/step - loss: 0.2979 - val_loss: 0.2727
Epoch 7/20
32/32 [==============================] - 24s 743ms/step - loss: 0.2755 - val_loss: 0.2817
Epoch 8/20
32/32 [==============================] - 24s 749ms/step - loss: 0.2536 - val_loss: 0.3004
Epoch 9/20
32/32 [==============================] - 24s 743ms/step - loss: 0.2357 - val_loss: 0.3119
Epoch 10/20
32/32 [=============================

In [8]:
# Saving history
import pickle
with open('./trainHistoryDict', 'wb') as file_pi:
    pickle.dump(history_fit, file_pi)

## Evaluating the training and dev losses

In [9]:
#import pickle
#with open('./trainHistoryDict', 'rb') as file_pi:
#    history_fit = pickle.load(file_pi)

In [10]:
# Calculating and printing root mean square error of fitting
rms_train = np.sqrt(history_fit['loss'])
rms_dev = np.sqrt(history_fit['val_loss'])

In [11]:
import matplotlib.pyplot as plt
plt.plot(rms_train)
plt.plot(rms_dev)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

<Figure size 640x480 with 1 Axes>

In [12]:
# Determining the best epoch
#print('RMS error of fitting:', rms_error_fit)